# 1. 문제 정의
- 집값을 예측해보자
- 회귀 문제로 접근
- 집 관련된 데이터로 집 가격 예측하기

# 2. 데이터 수집

In [1]:
import pandas as pd
data = pd.read_csv('./data/Boston_house.csv')
data.head()

,AGE,B,RM,CRIM,DIS,INDUS,LSTAT,NOX,PTRATIO,RAD,ZN,TAX,CHAS,Target
0,65.2,396.90,6.575,0.00632,4.0900,2.31,4.98,0.538,15.3,1,18.0,296,0,24.0
1,78.9,396.90,6.421,0.02731,4.9671,7.07,9.14,0.469,17.8,2,0.0,242,0,21.6
2,61.1,392.83,7.185,0.02729,4.9671,7.07,4.03,0.469,17.8,2,0.0,242,0,34.7
3,45.8,394.63,6.998,0.03237,6.0622,2.18,2.94,0.458,18.7,3,0.0,222,0,33.4
4,54.2,396.90,7.147,0.06905,6.0622,2.18,5.33,0.458,18.7,3,0.0,222,0,36.2


# 3. 데이터 전처리

In [2]:
X = data.loc[:,:'CHAS']
y = data.loc[:,'Target']

In [3]:
X.info

<bound method DataFrame.info of       AGE       B     RM     CRIM     DIS  INDUS  LSTAT    NOX  PTRATIO  RAD  \
0    65.2  396.90  6.575  0.00632  4.0900   2.31   4.98  0.538     15.3    1   
1    78.9  396.90  6.421  0.02731  4.9671   7.07   9.14  0.469     17.8    2   
2    61.1  392.83  7.185  0.02729  4.9671   7.07   4.03  0.469     17.8    2   
3    45.8  394.63  6.998  0.03237  6.0622   2.18   2.94  0.458     18.7    3   
4    54.2  396.90  7.147  0.06905  6.0622   2.18   5.33  0.458     18.7    3   
..    ...     ...    ...      ...     ...    ...    ...    ...      ...  ...   
501  69.1  391.99  6.593  0.06263  2.4786  11.93   9.67  0.573     21.0    1   
502  76.7  396.90  6.120  0.04527  2.2875  11.93   9.08  0.573     21.0    1   
503  91.0  396.90  6.976  0.06076  2.1675  11.93   5.64  0.573     21.0    1   
504  89.3  393.45  6.794  0.10959  2.3889  11.93   6.48  0.573     21.0    1   
505  80.8  396.90  6.030  0.04741  2.5050  11.93   7.88  0.573     21.0    1   

       

In [4]:
# 데이터 스케일링(Data scaling)
# 데이터의 범위를 정규화하기
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
X_robust = robust.fit_transform(X)
# 데이터를 변화실 범위의 값을 알아야헤서 학습의 단계가 있음

# 데이터 스케일링 언제 쓰나요?
# 모델중에서 거리 기반으로 학습하는 모델 (KNN)등을 사용할 때 높은 확률로 도움이 된다(성능 개선)
# 가중치 기반 학습하는 모델, SGD 등을 사용할땐 속도 개선에 도움이 된다.

# 일단 써보고 결과 좋아지면 다른것도 써본다 !
# 머신러닝 모델 뭐 써야할지 모르겠어요 > 아는거 다 써보고 결과 좋은거 선텍

In [5]:
X.head()

,AGE,B,RM,CRIM,DIS,INDUS,LSTAT,NOX,PTRATIO,RAD,ZN,TAX,CHAS
0,65.2,396.90,6.575,0.00632,4.0900,2.31,4.98,0.538,15.3,1,18.0,296,0
1,78.9,396.90,6.421,0.02731,4.9671,7.07,9.14,0.469,17.8,2,0.0,242,0
2,61.1,392.83,7.185,0.02729,4.9671,7.07,4.03,0.469,17.8,2,0.0,242,0
3,45.8,394.63,6.998,0.03237,6.0622,2.18,2.94,0.458,18.7,3,0.0,222,0
4,54.2,396.90,7.147,0.06905,6.0622,2.18,5.33,0.458,18.7,3,0.0,222,0


In [6]:
X_robust

array([[-0.25076453,  0.26190191,  0.49661247, ...,  1.44      ,
        -0.0878553 ,  0.        ],
       [ 0.0285423 ,  0.26190191,  0.28794038, ...,  0.        ,
        -0.22739018,  0.        ],
       [-0.3343527 ,  0.06667466,  1.32317073, ...,  0.        ,
        -0.22739018,  0.        ],
       ...,
       [ 0.27522936,  0.26190191,  1.0399729 , ...,  0.        ,
        -0.14728682,  0.        ],
       [ 0.24057085,  0.09641444,  0.79336043, ...,  0.        ,
        -0.14728682,  0.        ],
       [ 0.06727829,  0.26190191, -0.24186992, ...,  0.        ,
        -0.14728682,  0.        ]])

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# 데이터 확장
# 가지고 있는 개념은 활용해서 새로운 데이터를 만들어내는 것
# 컬럼끼리 곱하기
col = X.columns
for col1 in col: # 13개의 컬럼 반복
    for col2 in col: # 13개의 컬럼
        # 만들어질 컬럼 이름
        X[col1+'X'+col2] = X[col1]*X[col2]
        #컬럼이름           컬럼 값

In [9]:
X.shape

(506, 182)

# 4. 탐색적 데이터 분석
- 생략

# 5. 모델 선택 및 하이퍼 파라미터 튜닝

In [10]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [11]:
#데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size = 0.3,
                                                  random_state = 4)

In [12]:
from sklearn.linear_model import SGDRegressor
# 경사하강법에서 오차를 반영하는 정도
# eta0 : 학습률, 기본값 0.01
sgd = SGDRegressor(eta0 = 0.0000000000001)

# 6.학습

In [13]:
lr.fit(X_train,y_train)

LinearRegression()

In [14]:
sgd.fit(X_train, y_train)

SGDRegressor(eta0=1e-13)

# 7.평가

In [15]:
sgd.score(X_train,y_train)

0.22234055109841655

In [16]:
sgd.score(X_test,y_test)

0.15527454002182461

In [17]:
lr.score(X_train,y_train)

0.9280564047275944

In [18]:
lr.score(X_test,y_test)

0.8581336375476829

In [19]:
# 회귀 평가 지표
## 회귀는 완벽한 정답이 있긴한데, 분류에 비해 의미가 낮음
## 예측 : 1억1천만원 , 실제값 1억1천1십만원 , 틀렸다.
## 회귀는 얼마나 근접하게 대답했느냐가 중요
## 따라서 완벽히 정답을 맞췄느냐가 중요하지 않으니깐 정확도라는 지표가 없음
## 기본적으로 사용하는 지표 : MSE(평균 제곱 오차)
## 오차가 작을수록 좋은 모델, 오차가 0에 가까울수록 좋은 모델

## 정답 : 10000 , 예측 : 11000 , 오차 : 1000
## 정답 : 1000000 , 예측 : 10010000 , 오차 : 1000
## 오차의 크기만으로 모델의 성능을 평가하기는 힘들다, 값의 크기도 같이 고려되어야한다
# R2 score - = -1 - (오차 제곱의 합) / (표준 편차의 합)
## sklearn에서 회귀모델은 R2 score 사용
## 오찬의 크기와 값의 크기를 기반으로 해서 오차를 최대값이 1이 되도록 값을 수정
## - 값 : 매우 안 좋은 값
## 0~1 : 정확도 처럼 해석 / 0.7 이면 70% 확률로 정답이다

In [20]:
# 경사 하강법 - 선형회귀모델 (SGDRegressor)
## 처음엔 랜덤한 선형함수 하나를 고른다
## 점진적으로 오차가 가장 작은 선형함수를 찾아가는 방법
## w 값의 변화에 따라 오차가 변함, w값이 변한다 = 선형함수가 변한다, y = wx + b
## W 값을 어떤식으로 변화시켜야 오차가 작아질까?
## > W 값을 미분해보면 어떤식으로 변화해야하는지 알 수 있다
## > w 값을 미분했을때 나오는 기울기가 작아지는 방향으로 이동해야 한다
## >>> 기울기가 작아진다 = 경사 하강 법
## 점진적으로 이동시킬때 얼마만큼의 간격으로 이동시킬 것인지를 결정 해줘야함
## 너무 조금씩 이동시키면 이동횟수가 늘어나서 비효율적
## 너무 많이씩 이동시키면 오차가 커지면서 발산가능성이 생김

In [21]:
# 선형회귀
# 1. 데이터를 잘 설명하는 선형 함수를 찾아야한다.
# 2. MSE가 가장 작은 선형함수가 데이터를 잘 설명하는 것이다.
# 3. 선형함수중 몇개를 뽑아와서 그중에서 MSR를 계산하자.
#   1) 수학 공식을 이용한 모델 > Line
#   2) 경사하강법 > SGDRegressor

# 분류와 회귀의 차이점
1. 사용하는 모덿
2. 평가방법

선형 회귀  모델
-중요한 이유 : 딥러닝의 기초가 되는 모 델이다
-딥러닝의 기초 모델은 여러개의 선형 회귀
- 규칙을 찾는 방법 : 데이터를 표현할 수 있는 선형 함수 찾기
- 입력된 데이터를 설명할 수 있는 선형 함수 찾기(y= 10x)
- 입력된 데이터의 갯수에 따라서 선형 함수 공식의 길이가 달라진다

# 핵심
 1. 입력된 데이터를 설명할 수 있는 선형 함수 찾기
 2. '잘' = 오차가 적은, 여러 선형 함수 중에서 오차가 제일 적은 선형 함수 찾기
    - 평균 제곱 오차 : Mean Squared Error, MSE 오차의 크기를 계산하기 위해서 사용
 3. 이론적으로 선형함수의 갯수는 무한대
- 무한대를 계산할 수 없으니 선형함수를 몇개만 고르자
    - 수학 공식을 이용한 해석적 방법 : LinearRegression(머신러닝 모델)
      - 오차가 가장 작은 선형함수 한개를 찾는 공식(미완성)
      - 장점 : 한번에 오차가 가장 작은 선을 찾아서 시간이 짧게 걸림
      - 단점 : 틀렸을때 수정 불가
    - 경사하강법 : SDGRegressor(머신러닝 모델)
      - 점진적으로 오차가 작은 선형함수를 찾아가는 방법
      - 장점 : 여러번에 걸쳐 선형함수를 찾아가기 때문에 틀려도 수정이 가능
      - 단점 : 시간이 오래 걸림
- 입력된 데이터를 잘 설명하는 선형함수를 찾기위해서, 1번 또는 2번의 방법을 사용해서
  몇개의 선형함수를 고르고 MSE를 계산해서 오차가 가장 작은 선형 함수가 입력된 데이터를
  가장 잘 설명하는 선형함수이다. 이루어짐 